In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Consumption.csv')
df

,Unnamed: 0,yd,ce
0,1,59505.0,57168
1,2,59717.4,55464
2,3,59039.1,56332
3,4,61342.6,55836
4,5,60544.6,54488
...,...,...,...
195,196,390576.6,359372
196,197,391381.5,363896
197,198,389502.6,364428
198,199,388282.1,366224


In [3]:
# дельта - берем измерение и вычитаем предыдущее измерение
df.diff()

,Unnamed: 0,yd,ce
0,NaN,NaN,NaN
1,1.0,212.4,-1704.0
2,1.0,-678.3,868.0
3,1.0,2303.5,-496.0
4,1.0,-798.0,-1348.0
...,...,...,...
195,1.0,2336.8,24.0
196,1.0,804.9,4524.0
197,1.0,-1878.9,532.0
198,1.0,-1220.5,1796.0


In [5]:
df['log_yd']=np.log(df['yd'])
df['log_ce']=np.log(df['ce'])
df

,Unnamed: 0,yd,ce,log_yd,log_ce
0,1,59505.0,57168,10.993816,10.953750
1,2,59717.4,55464,10.997379,10.923489
2,3,59039.1,56332,10.985955,10.939018
3,4,61342.6,55836,11.024230,10.930174
4,5,60544.6,54488,11.011136,10.905736
...,...,...,...,...,...
195,196,390576.6,359372,12.875379,12.792113
196,197,391381.5,363896,12.877438,12.804623
197,198,389502.6,364428,12.872626,12.806084
198,199,388282.1,366224,12.869487,12.811000


In [7]:
df.diff().dropna() #убираем все строки NaN

,Unnamed: 0,yd,ce,log_yd,log_ce
1,1.0,212.4,-1704.0,0.003563,-0.030260
2,1.0,-678.3,868.0,-0.011423,0.015529
3,1.0,2303.5,-496.0,0.038275,-0.008844
4,1.0,-798.0,-1348.0,-0.013094,-0.024438
5,1.0,-220.0,-812.0,-0.003640,-0.015015
...,...,...,...,...,...
195,1.0,2336.8,24.0,0.006001,0.000067
196,1.0,804.9,4524.0,0.002059,0.012510
197,1.0,-1878.9,532.0,-0.004812,0.001461
198,1.0,-1220.5,1796.0,-0.003138,0.004916


In [9]:
model = smf.ols(data=df.diff().dropna(), formula='log_yd~log_ce').fit()
model.params.round(2)

Intercept    0.01
log_ce       0.37
dtype: float64

In [14]:
res0 = model.resid # 
res1 = res0.drop(labels=[1], axis=0) # labels = 1 - ту СТРОКУ, которая содержит индекс 1, надо удалить
res1

2     -0.023126
3      0.035603
4     -0.009987
5     -0.004025
6      0.019736
         ...   
195    0.000028
196   -0.008526
197   -0.011302
198   -0.010909
199   -0.009642
Length: 198, dtype: float64

In [16]:
res2 =res1.drop(labels=[2], axis=0)
res3 =res2.drop(labels=[3], axis=0)

In [21]:
# создаем новый столбец
DF = df.diff().dropna()
DF['res0'] = res0.reset_index(drop=True)
DF['res1'] = res1.reset_index(drop=True)
DF['res2'] = res2.reset_index(drop=True)
DF['res3'] = res3.reset_index(drop=True)
DF.dropna(inplace=True) # удаляем строки, которые содержат Nan'ы
DF

,Unnamed: 0,yd,ce,log_yd,log_ce,res0,res1,res2,res3
1,1.0,212.4,-1704.0,0.003563,-0.030260,-0.023126,0.035603,-0.009987,-0.004025
2,1.0,-678.3,868.0,-0.011423,0.015529,0.035603,-0.009987,-0.004025,0.019736
3,1.0,2303.5,-496.0,0.038275,-0.008844,-0.009987,-0.004025,0.019736,-0.020565
4,1.0,-798.0,-1348.0,-0.013094,-0.024438,-0.004025,0.019736,-0.020565,-0.000112
5,1.0,-220.0,-812.0,-0.003640,-0.015015,0.019736,-0.020565,-0.000112,-0.026773
...,...,...,...,...,...,...,...,...,...
191,1.0,2128.3,3196.0,0.005497,0.009014,0.003194,-0.009434,-0.014963,0.000028
192,1.0,3283.7,-692.0,0.008422,-0.001945,-0.009434,-0.014963,0.000028,-0.008526
193,1.0,-697.9,1636.0,-0.001784,0.004592,-0.014963,0.000028,-0.008526,-0.011302
194,1.0,-2610.5,2236.0,-0.006701,0.006242,0.000028,-0.008526,-0.011302,-0.010909


In [32]:
mdl=smf.ols(data=DF, formula='res0~res1+res2+res3+log_ce').fit()
mdl.rsquared.round(2)

0.11

In [33]:
R2 = mdl.rsquared
round(R2*(len(DF)-3),2)

20.74

In [35]:
Hi2 = stats.chi2.ppf(1-0.01, 3)
Hi2.round(2)

11.34

In [37]:
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/icecream.csv')
df

,obs,cons,price,income,temp,week
0,1,0.386,0.270,78,41,1
1,2,0.374,0.282,79,56,2
2,3,0.393,0.277,81,63,3
3,4,0.425,0.280,80,68,4
4,5,0.406,0.272,76,69,5
5,6,0.344,0.262,78,65,6
6,7,0.327,0.275,82,61,7
7,8,0.288,0.267,79,47,8
8,9,0.269,0.265,76,32,9
9,10,0.256,0.277,79,24,10


In [46]:
model = smf.ols(data=df, formula = 'cons ~ income + price + temp').fit(cov_type='HC3')
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cons   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     16.84
Date:                Wed, 10 May 2023   Prob (F-statistic):           2.78e-06
Time:                        11:47:59   Log-Likelihood:                 58.619
No. Observations:                  30   AIC:                            -109.2
Df Residuals:                      26   BIC:                            -103.6
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1973      0.318      0.620      0.535      -0.426       0.821
income         0.0033      0.001      2.573      0.010       0.001       0.006
price         -1.0444      1.010     -1.034      0.301      -3.024       0.935
temp           0.0035      0.001      6.866      0.000       0.002       0.004
==============================================================================
Omnibus:                        0.565   Durbin-Watson:                   1.021
Prob(Omnibus):                  0.754   Jarque-Bera (JB):                0.047
Skew:                           0.038   Prob(JB):                        0.977
Kurtosis:                       3.179   Cond. No.                     1.27e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 1.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [47]:
model = smf.ols(data=df, formula = 'cons ~ income + price + temp').fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cons   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     22.17
Date:                Wed, 10 May 2023   Prob (F-statistic):           2.45e-07
Time:                        11:47:59   Log-Likelihood:                 58.619
No. Observations:                  30   AIC:                            -109.2
Df Residuals:                      26   BIC:                            -103.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1973      0.270      0.730      0.472      -0.358       0.753
income         0.0033      0.001      2.824      0.009       0.001       0.006
price         -1.0444      0.834     -1.252      0.222      -2.759       0.671
temp           0.0035      0.000      7.762      0.000       0.003       0.004
==============================================================================
Omnibus:                        0.565   Durbin-Watson:                   1.021
Prob(Omnibus):                  0.754   Jarque-Bera (JB):                0.047
Skew:                           0.038   Prob(JB):                        0.977
Kurtosis:                       3.179   Cond. No.                     1.27e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""